In [18]:
#https://github.com/garywu/google-compute-engine-selenium
# install chromium, its driver, and selenium
# !sudo apt install chromium-chromedriver
# !sudo cp /usr/lib/chromium-browser/chromedriver /usr/bin
# !pip install selenium
# set options to be headless, ..
import time
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver= webdriver.Chrome('chromedriver',options=options)


# open it, go to a website, and get results
# wd = webdriver.Chrome('chromedriver',options=options)
# wd.get("https://www.website.com")
# print(wd.page_source)  # results

In [19]:

from google.cloud import storage

def upload_to_bucket(blob_name, path_to_file):
    """ Upload data to a bucket""" #doc string in function

    # Explicitly use service account credentials by specifying the private key
    # file.
    storage_client = storage.Client.from_service_account_json(
        'auth.json')

    #print(buckets = list(storage_client.list_buckets())

    bucket = storage_client.get_bucket('urbankiddo')
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(path_to_file)

    #returns a public url
    return blob.public_url

In [3]:
import csv


f = open("oildesign.csv", "w", encoding='utf-8')
writer = csv.DictWriter(f, fieldnames=["name","brand","gender","category","size","price","url","img_url","article_no","blob_url"])
writer.writeheader()
f.close()

import pandas as pd
df=pd.read_csv("oildesign.csv", encoding='utf-8')
df.head()

,name,brand,gender,category,size,price,url,img_url,article_no,blob_url


In [45]:
categorys=['swimwear','bodysuits','boots','bedtextile','fleece','scarves','gloves','gowns and jackets','skirts','the','dress',
          'costumes','hats','skirts','overalls','shorts','sleepwear','sportswear','sweaters and knitwear','underwear','underwear','tops']
main_urls=['https://www.oiidesign.se/barnklader/s/1?p=561737',
          'https://www.oiidesign.se/barnklader/s/1?p=557799',
          'https://www.oiidesign.se/barnklader/s/1?p=558527',
          'https://www.oiidesign.se/barnklader/s/1?p=569660',
          'https://www.oiidesign.se/barnklader/s/1?p=562736',
          'https://www.oiidesign.se/barnklader/s/1?p=562208',
          'https://www.oiidesign.se/barnklader/s/1?p=562836',
          'https://www.oiidesign.se/barnklader/s/1?p=561693',
          'https://www.oiidesign.se/barnklader/s/1?p=561847',
          'https://www.oiidesign.se/barnklader/s/1?p=565739',
          'https://www.oiidesign.se/barnklader/s/1?p=557879',
          'https://www.oiidesign.se/barnklader/s/1?p=637242',
          'https://www.oiidesign.se/barnklader/s/1?p=558220',
          'https://www.oiidesign.se/barnklader/s/1?p=560540',
          'https://www.oiidesign.se/barnklader/s/1?p=563724',
          'https://www.oiidesign.se/barnklader/s/1?p=561685',
          'https://www.oiidesign.se/barnklader/s/1?p=563815',
          'https://www.oiidesign.se/barnklader/s/1?p=788007',
          'https://www.oiidesign.se/barnklader/s/1?p=560608',
          'https://www.oiidesign.se/barnklader/s/1?p=559052',
          'https://www.oiidesign.se/barnklader/s/1?p=623108',
          'https://www.oiidesign.se/barnklader/s/1?p=558949'
          ]

In [46]:
categorys=categorys[21:]
main_urls=main_urls[21:]



In [47]:
categorys

['tops']

In [5]:
!mkdir oildesign

mkdir: cannot create directory ‘oildesign’: File exists


In [48]:
from bs4 import BeautifulSoup
import time
import requests

gender='unisex'

for c,main_url in enumerate(main_urls):
    category=categorys[c]
    page_url=main_url+'&viewall=true'
    html_doc=requests.get(page_url).content.decode()
    soup = BeautifulSoup(html_doc, 'html.parser')
    products=soup.find("div",attrs={"id":"products"})
    print(page_url)
    print(category)

    for product in products.find_all("article"):
        article_no=product['data-id']
        url='https://www.oiidesign.se'+product.find("a")['href']
        img_url='https://www.oiidesign.se'+product.find("img")['src']
        name=product.find("p",attrs={"class":"name"}).text
        brand=product.find("p",attrs={"class":"brand"}).text
        price=product.find("p",attrs={"class":"price"}).text
        
        ac='oildesign'+'/'+article_no+str('.jpg')
        rr = requests.get(img_url, allow_redirects=True)
        open(ac, 'wb').write(rr.content)

        blob_name=str('oildesign')+str('/')+str(article_no)
        path_to_file='oildesign/'+article_no+str('.jpg')
        blob_url=upload_to_bucket(blob_name, path_to_file)
        
        driver= webdriver.Chrome('chromedriver',options=options)
        driver.get(url)
        driver.set_window_size(1920,1080)

        html_doc2=driver.page_source
        driver.quit()
        soup2 = BeautifulSoup(html_doc2, 'html.parser')
        sizes=soup2.find_all("li",attrs={"class":"option"})
        for sz in sizes:
            size=sz.text.strip()
            with open("oildesign.csv", "a", encoding='utf-8') as f:
                writer = csv.writer(f)
                writer.writerow([name,brand,gender,category,size,price,url,img_url,article_no,blob_url])

            

https://www.oiidesign.se/barnklader/s/1?p=558949&viewall=true
tops


In [ ]:
page_url

In [ ]:

driver= webdriver.Chrome('chromedriver',options=options)
driver.get('https://www.oiidesign.se/black-red-colourblock-water-reactive-upf-50-swim-shorts/p/144491')
driver.set_window_size(1920,1080)


In [ ]:
html_doc2=driver.page_source
driver.quit()
soup2 = BeautifulSoup(html_doc2, 'html.parser')
sizes=soup2.find_all("li",attrs={"class":"option"})
for sz in sizes:
    size=sz.text.strip()
    print(size)

In [ ]:
size.strip()

In [ ]:
soup2.find("div",attrs={"class":"slct id-slct closed"})

In [49]:
from google.cloud import storage
storage_client = storage.Client.from_service_account_json('auth.json')
bucket = storage_client.get_bucket('urbankiddo')
blob = bucket.blob('oildesign.csv')
blob.upload_from_filename('oildesign.csv')
blob.public_url

'https://storage.googleapis.com/urbankiddo/oildesign.csv'